#### Objective of this notebook is to automate geocoding of inventor addresses from patent applications data using Google Maps API

In [ ]:
pip install -U googlemaps

In [ ]:
## Listing all packages that will be used in this notebook
import googlemaps
import pandas as pd
import numpy
import os

from datetime import datetime

In [ ]:
## Entering the API key
gmaps = googlemaps.Client(key= 'put your key here') # Replace 'put your key here' with your Google maps key

In [ ]:
# Changing the working Directory
os.chdir('put the directory path here')

I have three different excel files containing raw address data of inventors, each for decades 1990, 2000, 2010. Following gives example for one of them. By changing the name of files, the process can be repeated for each one of them- the benefit of automation!

In [ ]:
## Specifying input and output files 
start_year=1990 ## change to 2000, 2010 for the other two files

INPUT_FILE= 'patent_address_cleaned_Domestic{}.xlsx'.format(start_year)

OUTPUT_FILE= os.path.join(os.getcwd(), 'AddressData_{}.xls'.format(start_year))

In [ ]:
## Feeding in the data
address_data = pd.read_excel(INPUT_FILE)

## Creating a new DataFrame that will store the geocoded data
add_data = pd.DataFrame(columns = ["formatted_address","street_number","route","sublocality_level_3",
                                   "sublocality_level_2","sublocality_level_1","locality",
                                   "administrative_area_level_2","administrative_area_level_1",
                                   "country","postal_code", "lat", "lng", "str_raw", "input_check"])
col_names= list(add_data.columns)

In [ ]:
## Defining a function that will put the results from geocoded addresses in a proper format
def cur_obs(current_address):
    geocode_result = gmaps.geocode(current_address) ## gmaps.geocode() yields the geocoded address. 
    # Objective is to put the output of this data in a dataframe
     
    this_obs= pd.DataFrame(columns = col_names) # Initiates an obs that will contain the results from geocode_result
    this_obs['input_check']= [current_address]
    ## Getting the lat and long
    geometry= geocode_result[0].get('geometry')
    obs= geometry.get('location')
    this_obs['lat']= [obs.get('lat')]
    this_obs['lng']= [obs.get('lng')]
    
    ## other components of address
    this_obs['formatted_address'] = [geocode_result[0].get('formatted_address')]
    comps= geocode_result[0].get('address_components')
    this_obs['str_raw'] = [comps]
    for i in range(len(comps)):
        if 'street_number' in comps[i].get('types'):
            this_obs['street_number']= [comps[i].get('long_name')]
        if 'route' in comps[i].get('types'):
            this_obs['route']= [comps[i].get('long_name')]
        if 'sublocality_level_1' in comps[i].get('types'):
            this_obs['sublocality_level_1']= [comps[i].get('long_name')]
        if 'sublocality_level_2' in comps[i].get('types'):
            this_obs['sublocality_level_2']= [comps[i].get('long_name')]
        if 'sublocality_level_3' in comps[i].get('types'):
            this_obs['sublocality_level_3']= [comps[i].get('long_name')]
        if 'locality' in comps[i].get('types'):
            this_obs['locality']= [comps[i].get('long_name')]
        if 'administrative_area_level_1' in comps[i].get('types'):
            this_obs['administrative_area_level_1']= [comps[i].get('long_name')]
        if 'administrative_area_level_2' in comps[i].get('types'):
            this_obs['administrative_area_level_2']= [comps[i].get('long_name')]
        if 'country' in comps[i].get('types'):
            this_obs['country']= [comps[i].get('long_name')]
        if 'postal_code' in comps[i].get('types'):
            this_obs['postal_code']= [comps[i].get('long_name')]
    
    return(this_obs)

In [ ]:
## The input file has raw addresses in column named "add"
## Looping over thsi column and appending the resulst in the dataframe called add_data
for n in range(len(address_data['add'])):
    curr_add= address_data['add'][n]
    
    current_obs= cur_obs(current_address= curr_add) ## applying our cur_obs function
    
    add_data= add_data.append(current_obs)

# Transferring data to output file
add_data.to_excel(OUTPUT_FILE) 